# Library for sequential neural networks

In [616]:
import math
import random as rnd
import numpy as np
from enum import Enum

### Classes

#### Neuron

In [617]:
class Neuron:
    def __init__(self, id):
        self.id = id
        self.output = 0
        self.bias = 0 if self.id[0] == 0 else round((rnd.random() - 0.5) * 5, 2) # if layer id is 0, it is an input layer
    
    def propagate(self, prev_layer_neurons, current_layer_weights, current_layer_activation_function):
        net_input = 0

        for n in prev_layer_neurons:
            net_input += n.output * current_layer_weights[n.id[1]][self.id[1]]
            
        net_input += self.bias
        self.output = round(self.activate(net_input, current_layer_activation_function), 2) # ToDo: How does the previous activation play a role?

    def activate(self, input, activation):
        match activation:
            case 'identity':
                return input
            case 'relu':
                return max(0, input)
            case 'binary_step':
                return 0 if input < 0 else 1
            case 'sigmoid':
                return 1 / (1 + math.exp(-input))
            case 'tanh':
                return math.tanh(input)


#### Layer

In [618]:
class Layer:
    def __init__(self, id, num_of_neurons, activation = 'identity', prev_layer = None):
        self.id = id
        self.num_of_neurons = num_of_neurons
        self.activation = activation if activation in ['identity', 'relu', 'binary_step', 'sigmoid', 'tanh'] else 'identity'

        self.neurons = []

        for i in range(self.num_of_neurons):
            self.neurons.append(Neuron((self.id, i), ))
        
        if self.id > 0:
            self.weights = np.zeros((prev_layer.num_of_neurons, self.num_of_neurons))
            
            for i in range(len(self.weights)):
                for j in range(len(self.weights[0])):
                    self.weights[i][j] = round(rnd.random(), 2)

    def add_neuron(self):
        self.neurons.append(Neuron((self.id, len(self.neurons))))

### Network

In [619]:
class Network_Model:
    def __init__(self, id):
        self.id = id
        self.layers = []

    def add_layer(self, num_of_neurons, activation = 'identity'):
        if num_of_neurons < 1:
            print('Number of neurons has to be larger or equal to 0!')
            return
        
        if activation not in ['identity', 'relu', 'binary_step', 'sigmoid', 'tanh']:
            print(f'Activation function "{activation}" does not exist!')
            return

        if len(self.layers) > 0:
            self.layers.append(Layer(len(self.layers), num_of_neurons, activation, self.layers[-1]))
        else:
            self.layers.append(Layer(len(self.layers), num_of_neurons, activation))

    def plot_network(self):
        for l in self.layers:
            s = '|'
            for n in l.neurons:
                s += ' ' + str(n.id) + ', ' + str(n.output) + ' |'
            print(s)
    
    def learn(self):
        pass

    def predict(self, input):
        # write input to first layer
        if len(input) != len(self.layers[0].neurons):
            print('Not enough input values!')
            return

        for i, n in enumerate(self.layers[0].neurons):
            n.output = input[i]
        
        # propagate
        for l in self.layers:
            if l.id == 0: continue # prevent input neurons from propagating

            for n in l.neurons:
                n.propagate(self.layers[l.id - 1].neurons, l.weights, l.activation)
    
        for n in self.layers[-1].neurons:
            print(n.output)

    @staticmethod
    def normalize(input, max_value):
        normalized = []
        
        for i in input:
            normalized.append(round(i / max_value, 2))
        return normalized

### Instanciating a network

In [620]:
mdl = Network_Model(0)
mdl.add_layer(5)
mdl.add_layer(10, activation='relu')
mdl.add_layer(1, activation='sigmoid')

In [621]:
mdl.plot_network()

| (0, 0), 0 | (0, 1), 0 | (0, 2), 0 | (0, 3), 0 | (0, 4), 0 |
| (1, 0), 0 | (1, 1), 0 | (1, 2), 0 | (1, 3), 0 | (1, 4), 0 | (1, 5), 0 | (1, 6), 0 | (1, 7), 0 | (1, 8), 0 | (1, 9), 0 |
| (2, 0), 0 |


In [622]:
input = [1, 0, 0, 0, 0]
mdl.predict(input)

0.89


In [623]:
mdl.plot_network()

| (0, 0), 1 | (0, 1), 0 | (0, 2), 0 | (0, 3), 0 | (0, 4), 0 |
| (1, 0), 1.54 | (1, 1), 0.2 | (1, 2), 0 | (1, 3), 1.1 | (1, 4), 0 | (1, 5), 1.85 | (1, 6), 0 | (1, 7), 0 | (1, 8), 0 | (1, 9), 0 |
| (2, 0), 0.89 |
